In [18]:
# Import all needed libraries

# ! pip install google-cloud-bigquery
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
#from config import db_password

In [19]:
# GCP Authentication
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="client_secret.json"

In [20]:
client = bigquery.Client()

In [21]:
# Exploring raw dataset
hn_dataset_ref = client.dataset('stackoverflow', project='bigquery-public-data')
hn_dataset_ref

DatasetReference('bigquery-public-data', 'stackoverflow')

In [22]:
type(hn_dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [23]:
hn_dset = client.get_dataset(hn_dataset_ref)
hn_dset

Dataset(DatasetReference('bigquery-public-data', 'stackoverflow'))

In [24]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [25]:
# Determine available data tables
[x.table_id for x in client.list_tables(hn_dset)]

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [26]:
# Exploring data in posts_questions
hn_full = client.get_table(table='bigquery-public-data.stackoverflow.posts_questions')
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-01-01'
        ORDER BY accepted_answer_id
        LIMIT 10"""
    )

results = query_job.result() 
results_list = [row for row in results]
cols = [field.name for field in results.schema]
posts_questions = pd.DataFrame.from_records(results_list, columns=cols)
posts_questions

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,65526420,How to check if values in individiual rows of ...,<p>Suppose I have the following data.table:</p...,65526457,2,2,None,2021-01-01 00:05:46.310000+00:00,2.0,2021-01-01 02:15:42.223000+00:00,NaT,None,NaN,None,6335637,None,1,2,r|data.table|rowwise,62
1,65526423,d3.js : Generating axis ticks for ordinal values,<p>I want to use ordinal scale in x-axis with ...,65526533,1,0,None,2021-01-01 00:06:09.007000+00:00,NaN,2021-01-01 16:25:58.447000+00:00,NaT,None,NaN,None,9574155,None,1,2,javascript|d3.js,48
2,65526490,Is there a C macro that replaces varied length...,<p>I want to be able to:</p>\n<pre><code>#defi...,65526541,1,1,None,2021-01-01 00:20:09.553000+00:00,0.0,2021-01-01 00:37:24.277000+00:00,NaT,None,NaN,None,12743240,None,1,2,c|macros,35
3,65526419,How can I construct my objects allocated throu...,<p>C++20 removed the <code>construct()</code> ...,65526554,2,0,None,2021-01-01 00:05:43.627000+00:00,NaN,2021-01-01 18:19:46.870000+00:00,2021-01-01 05:23:28.560000+00:00,None,65863.0,None,14694500,None,1,3,c++|std|c++20|allocator,351
4,65526523,Navigate from parent to child in react where e...,<p>In a React app with a parent and child elem...,65526577,1,1,None,2021-01-01 00:30:31.933000+00:00,NaN,2021-01-01 00:46:22.813000+00:00,2021-01-01 00:35:53.623000+00:00,None,8690857.0,None,3002584,None,1,2,reactjs|react-router|react-router-dom,117
5,65526451,How do I define a field in a subclass by stric...,<p>I'm trying to implement the UML diagram bel...,65526589,1,0,None,2021-01-01 00:11:17.787000+00:00,NaN,2021-01-01 10:22:51.790000+00:00,2021-01-01 10:22:51.790000+00:00,None,2458991.0,None,14634129,None,1,1,java|inheritance|uml,67
6,65526591,How to install Python 3.5.x on Ubuntu 18.04 LT...,<p>Simple question: How to install Python 3.5....,65526611,2,1,None,2021-01-01 00:49:45.320000+00:00,NaN,2021-01-01 09:48:41.703000+00:00,NaT,None,NaN,None,6305105,None,1,0,python|ubuntu,123
7,65526474,GDB: Displaying incorrect values in struct,<p>I'm trying to implement the <code>malloc</c...,65526637,1,0,None,2021-01-01 00:16:10.673000+00:00,NaN,2021-01-01 01:02:22.447000+00:00,2021-01-01 00:39:32.250000+00:00,None,1233251.0,None,11035194,None,1,1,struct|gdb|memory-address,34
8,65526590,Vue2 + laravel6 - Component implementation,<p>I just started using Vue2 with Laravel6 and...,65526639,1,0,None,2021-01-01 00:49:32.030000+00:00,NaN,2021-01-01 01:09:45.250000+00:00,2021-01-01 01:07:22.133000+00:00,None,11960598.0,None,14605909,None,1,0,laravel|vue.js|vuejs2|laravel-6,39
9,65526522,Mobile menu css,<p>What's the best way to\nachieve going from ...,65526667,2,0,None,2021-01-01 00:30:28.633000+00:00,NaN,2021-01-01 01:46:35.280000+00:00,NaT,None,NaN,None,6787542,None,1,0,css,39


In [27]:
# Extract data for May 2021 for posts with accepted ansers
query_job = client.query(
        """
        SELECT
        id, title, body, accepted_answer_id, creation_date, favorite_count, score, tags, view_count
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-05-01'
        ORDER BY accepted_answer_id"""
    )

results = query_job.result() 
results_list = [row for row in results]
cols = ['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date', 'q_favorite_count', 'q_score', 'q_tags', 'q_view_count']
posts_questions_df = pd.DataFrame.from_records(results_list, columns=cols)
posts_questions_df.sample(n=15)

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_favorite_count,q_score,q_tags,q_view_count
141274,68481027,Adding quotes around cell in formula?,<p>I am trying to create a formula that will h...,68482597,2021-07-22 07:47:25.353000+00:00,NaN,1,excel|excel-formula,31
177043,68772093,Can any one help me to decode Conditional(?) O...,<p>I have to decode below expression in SQL qu...,68772329,2021-08-13 12:09:13.317000+00:00,NaN,0,sql-server|ssis,20
84794,68031043,Efficient way to add sample information as new...,<p>I know how I can subset a data frame by sam...,68031115,2021-06-18 07:27:13.630000+00:00,NaN,0,r|random|tidyverse,24
57877,67812318,How do you use std::declval find the return va...,<p>I have a class <code>B</code> that takes a ...,67813664,2021-06-02 20:42:53.763000+00:00,NaN,0,c++|templates|types|decltype,69
11131,67438197,Indexing in for loops,<p>Beginner programming student here. I'm str...,67438375,2021-05-07 15:57:13.333000+00:00,NaN,1,r|loops|indexing,44
140606,68477321,Using a for-loop and range function vs a while...,<p>I'm looking for a function like range excep...,68477393,2021-07-21 22:51:49.713000+00:00,NaN,2,python|numpy|for-loop|while-loop|range,44
112287,68250683,Using Youtube's Javascript API with Angular,<p>I'm currently using the <code>@angular/yout...,68251210,2021-07-05 04:20:39.017000+00:00,1.0,1,angular,41
195032,68918863,"How to combine two columns into one in R, so t...",<p>I have the following dataset:</p>\n<pre><co...,68919984,2021-08-25 07:48:42.793000+00:00,NaN,0,r|transform|donut-chart,37
83091,68017118,Fatal error: Uncaught Error: Call to undefined...,<p>I try to get the 5 most-watched Movies from...,68017237,2021-06-17 10:02:14.167000+00:00,NaN,0,php|mysql|mysqli,37
167151,68691265,Sort by one column and count values of another...,<p>I have a dataframe as follows:</p>\n<pre><c...,68691452,2021-08-07 09:56:03.610000+00:00,NaN,3,python|pandas|dataframe,56


In [28]:
# Determine data types for posts_questions_df
posts_questions_df.dtypes

q_id                                int64
q_title                            object
q_body                             object
accepted_answer_id                  int64
q_creation_date       datetime64[ns, UTC]
q_favorite_count                  float64
q_score                             int64
q_tags                             object
q_view_count                        int64
dtype: object

In [29]:
# How many missing values in the DF by column
[[column,posts_questions_df[column].isnull().sum()] for column in posts_questions_df.columns]

[['q_id', 0],
 ['q_title', 0],
 ['q_body', 0],
 ['accepted_answer_id', 0],
 ['q_creation_date', 0],
 ['q_favorite_count', 193444],
 ['q_score', 0],
 ['q_tags', 0],
 ['q_view_count', 0]]

In [30]:
len(posts_questions_df)

210368

In [31]:
# Drop q_favorite_count column because too many null values
posts_questions_df = posts_questions_df.drop(columns='q_favorite_count')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_score', 'q_tags', 'q_view_count'],
      dtype='object')

In [32]:
# How many missing values in the DF by column
[[column,posts_questions_df[column].isnull().sum()] for column in posts_questions_df.columns]

[['q_id', 0],
 ['q_title', 0],
 ['q_body', 0],
 ['accepted_answer_id', 0],
 ['q_creation_date', 0],
 ['q_score', 0],
 ['q_tags', 0],
 ['q_view_count', 0]]

In [33]:
# Additional method to verify number of rows with zero response time (what's the possible explanation?)
q_score_explore_raw = posts_questions_df.groupby('q_score').count()
q_score_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count
q_score,,,,,,,
-11,1,1,1,1,1,1,1
-10,1,1,1,1,1,1,1
-9,6,6,6,6,6,6,6
-8,9,9,9,9,9,9,9
-7,15,15,15,15,15,15,15
...,...,...,...,...,...,...,...
103,1,1,1,1,1,1,1
125,1,1,1,1,1,1,1
153,1,1,1,1,1,1,1


In [34]:
# Bin score for more meaningful analysis
# Establish the bins.
q_score_bins = [-10, -1, 0, 10, 25, 50, 101]
q_score_group_names = ["Negative Score (<0)", "Zero Score (0)", "Low Score (0-10)", "Medium Score (10 - 25)", "High Score (25-50)", "Popular Score (50-101)"]

# Categorize score based on the bins.
posts_questions_df['q_score_tier'] = pd.cut(posts_questions_df['q_score'], q_score_bins, labels=q_score_group_names)

In [35]:
# explore scores
q_score_explore = posts_questions_df.groupby('q_score_tier').count()
q_score_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count
q_score_tier,,,,,,,,
Negative Score (<0),21235,21235,21235,21235,21235,21235,21235,21235
Zero Score (0),116565,116565,116565,116565,116565,116565,116565,116565
Low Score (0-10),72330,72330,72330,72330,72330,72330,72330,72330
Medium Score (10 - 25),169,169,169,169,169,169,169,169
High Score (25-50),46,46,46,46,46,46,46,46
Popular Score (50-101),15,15,15,15,15,15,15,15


In [38]:
posts_questions_df['target'] = posts_questions_df['q_score'].apply(lambda x: int(x != 0))
posts_questions_df['target'].value_counts()

0    116565
1     93803
Name: target, dtype: int64

In [39]:
# Bin score for more meaningful analysis (broader bin)

# Establish the bins.
q_score_bins = [-10, -1, 0, 101]
q_score_group_names = ["Negative Score (<0)", "Zero Score (0)", "Positive Score (>0)"]
# Categorize score based on the bins.
posts_questions_df['q_score_tier'] = pd.cut(posts_questions_df['q_score'], q_score_bins, labels=q_score_group_names)
# Bin scores
q_score_explore = posts_questions_df.groupby('q_score_tier').count()
q_score_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count,target
q_score_tier,,,,,,,,,
Negative Score (<0),21235,21235,21235,21235,21235,21235,21235,21235,21235
Zero Score (0),116565,116565,116565,116565,116565,116565,116565,116565,116565
Positive Score (>0),72560,72560,72560,72560,72560,72560,72560,72560,72560


In [40]:
# Verify whether there are redundant columns for score_tier
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_score', 'q_tags', 'q_view_count', 'q_score_tier', 'target'],
      dtype='object')

In [41]:
# Verify data in q_score_tier column
posts_questions_df.sample(n=10)

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_score,q_tags,q_view_count,q_score_tier,target
57088,67807251,Deserialize JSON string that supposed for obje...,<p>I need to serialize a string of JSON. But i...,67807453,2021-06-02 14:36:36.837000+00:00,0,c#|json|json.net,47,Zero Score (0),0
165143,68600014,How to invert the order in Matplotlib divergin...,<p>I have a dataframe with <code>ordered</code...,68674841,2021-07-31 07:19:33.590000+00:00,3,python|matplotlib|data-visualization,47,Positive Score (>0),1
89490,68066217,Get inverted value with single function in TER...,<p>Can you guys help me go through this:</p>\n...,68069327,2021-06-21 10:24:32+00:00,0,function|teradata,28,Zero Score (0),0
153614,68581659,I want my bot to process commands sent by othe...,"<p>This isn't something most people want, but ...",68581900,2021-07-29 19:06:38.660000+00:00,0,python|python-3.x|discord|discord.py,40,Zero Score (0),0
166606,68686872,Replace exact part of file name with Powershell,"<p>I tried the code found <a href=""https://sta...",68686905,2021-08-06 19:53:47.923000+00:00,1,powershell|file|file-rename|exact-match,53,Positive Score (>0),1
130353,68396723,Code acting like it is asynchronous when I don...,<p>I'm struggling to see why some code is beha...,68396950,2021-07-15 15:32:50.567000+00:00,0,javascript|firebase|google-cloud-firestore|fir...,48,Zero Score (0),0
54454,67768456,jqGrid jQuery Hexadecimal Colorpicker from dum...,<p>Spec : Trial based JqGrid 5.5 from &quot;ht...,67786641,2021-05-31 05:44:19.103000+00:00,0,jquery|jqgrid,29,Zero Score (0),0
15496,67439349,Validating the state of the slide toggle using...,<p>I'm just new to Cypress and I want to <stro...,67474337,2021-05-07 17:28:17.127000+00:00,0,javascript|node.js|testing|automation|cypress,173,Zero Score (0),0
193186,68904021,How to overcome hover issue in Cypress?,<p>I faced Cypress hover problem regarding acc...,68904742,2021-08-24 08:17:37.857000+00:00,1,javascript|jquery|css|cypress,25,Positive Score (>0),1
60121,67829069,Complex numbers in sympy,<p>I am using lcapy together with sympy and tr...,67831472,2021-06-03 21:49:33.817000+00:00,0,sympy,62,Zero Score (0),0


In [42]:
# Drop score column because data transformed to the score_tier column
posts_questions_df = posts_questions_df.drop(columns='q_score')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'target'],
      dtype='object')

In [43]:
# Determine length of title
posts_questions_df['q_title_char_count']= posts_questions_df['q_title'].str.len() #character count
posts_questions_df['q_title_word_count']= posts_questions_df['q_title'].str.split().str.len() #word count
posts_questions_df.head()

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_char_count,q_title_word_count
0,67341742,Is there any type of way to call a function in...,<p>sorry in advance for my lack of technical l...,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,38,Zero Score (0),0,95,20
1,67341817,Produce multiple models for CVC4 SMT queries,<p>Can I get multiple models for a query like ...,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,41,Zero Score (0),0,44,7
2,67341895,Why does shuffling array 'A' also shuffle arra...,<p>I'm trying to make a shuffled copy of an ar...,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,50,Positive Score (>0),1,97,17
3,67341936,Is there a simpler way to validate user input ...,<p>I am currently learning python and I was wo...,67341961,2021-05-01 00:41:17.673000+00:00,python,57,Positive Score (>0),1,86,15
4,67341921,How to Install vue3 in Laravel 8 without vue-s...,<p>I have install Laravel 8 fresh project. And...,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,42,Positive Score (>0),1,114,17


In [44]:
# Bin by title length (char count)
q_title_explore_char = posts_questions_df.groupby('q_title_char_count').count()
q_title_explore_char

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_word_count
q_title_char_count,,,,,,,,,,
15,65,65,65,65,65,65,65,65,65,65
16,120,120,120,120,120,120,120,120,120,120
17,145,145,145,145,145,145,145,145,145,145
18,193,193,193,193,193,193,193,193,193,193
19,271,271,271,271,271,271,271,271,271,271
...,...,...,...,...,...,...,...,...,...,...
146,106,106,106,106,106,106,106,106,106,106
147,113,113,113,113,113,113,113,113,113,113
148,132,132,132,132,132,132,132,132,132,132


In [45]:
# Bin title length by char count

# Establish the bins.
q_char_count_bins = [0, 50, 100, 150]
q_char_count_group_names = ["Short (0 - 50)", "Medium (50-100)", "Long (100-150)"]

# Categorize char length based on the bins.
posts_questions_df['q_title_char_count_bin'] = pd.cut(posts_questions_df['q_title_char_count'], q_char_count_bins, labels=q_char_count_group_names)

# Bin by char length
q_char_count_explore = posts_questions_df.groupby('q_title_char_count_bin').count()
q_char_count_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_char_count,q_title_word_count
q_title_char_count_bin,,,,,,,,,,,
Short (0 - 50),81247,81247,81247,81247,81247,81247,81247,81245,81247,81247,81247
Medium (50-100),117547,117547,117547,117547,117547,117547,117547,117542,117547,117547,117547
Long (100-150),11574,11574,11574,11574,11574,11574,11574,11573,11574,11574,11574


In [46]:
# Bin by title length (word count)
q_title_explore_word = posts_questions_df.groupby('q_title_word_count').count()
q_title_explore_word

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_char_count,q_title_char_count_bin
q_title_word_count,,,,,,,,,,,
1,18,18,18,18,18,18,18,18,18,18,18
2,334,334,334,334,334,334,334,334,334,334,334
3,2410,2410,2410,2410,2410,2410,2410,2410,2410,2410,2410
4,6756,6756,6756,6756,6756,6756,6756,6756,6756,6756,6756
5,12050,12050,12050,12050,12050,12050,12050,12050,12050,12050,12050
6,17744,17744,17744,17744,17744,17744,17744,17744,17744,17744,17744
7,21748,21748,21748,21748,21748,21748,21748,21747,21748,21748,21748
8,23927,23927,23927,23927,23927,23927,23927,23925,23927,23927,23927
9,23431,23431,23431,23431,23431,23431,23431,23431,23431,23431,23431


In [47]:
# Bin title length by word count

# Establish the bins.
q_title_word_count_bins = [0, 10, 20, 30, 40]
q_title_word_count_group_names = ["Short (0 - 10)", "Medium (10-20)", "Long (20-30)", "XL (30+)"]

# Categorize char length based on the bins.
posts_questions_df['q_title_word_count_bin'] = pd.cut(posts_questions_df['q_title_word_count'], q_title_word_count_bins, labels=q_title_word_count_group_names)

# Bin by char length
q_word_count_explore = posts_questions_df.groupby('q_title_word_count_bin').count()
q_word_count_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_char_count,q_title_word_count,q_title_char_count_bin
q_title_word_count_bin,,,,,,,,,,,,
Short (0 - 10),129640,129640,129640,129640,129640,129640,129640,129636,129640,129640,129640,129640
Medium (10-20),76349,76349,76349,76349,76349,76349,76349,76345,76349,76349,76349,76349
Long (20-30),4338,4338,4338,4338,4338,4338,4338,4338,4338,4338,4338,4338
XL (30+),41,41,41,41,41,41,41,41,41,41,41,41


In [48]:
# Clean up posts_questions_df
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'target',
       'q_title_char_count', 'q_title_word_count', 'q_title_char_count_bin',
       'q_title_word_count_bin'],
      dtype='object')

In [49]:
# Drop q_title_char_count and q_title_word_count columns because data transformed into bins
posts_questions_df = posts_questions_df.drop(columns='q_title_char_count')
posts_questions_df = posts_questions_df.drop(columns='q_title_word_count')
posts_questions_df.columns

Index(['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'q_creation_date',
       'q_tags', 'q_view_count', 'q_score_tier', 'target',
       'q_title_char_count_bin', 'q_title_word_count_bin'],
      dtype='object')

In [50]:
# Explore view count
q_views_explore_raw = posts_questions_df.groupby('q_view_count').count()
q_views_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_score_tier,target,q_title_char_count_bin,q_title_word_count_bin
q_view_count,,,,,,,,,,
6,6,6,6,6,6,6,6,6,6,6
7,19,19,19,19,19,19,19,19,19,19
8,42,42,42,42,42,42,42,42,42,42
9,78,78,78,78,78,78,78,78,78,78
10,177,177,177,177,177,177,177,177,177,177
...,...,...,...,...,...,...,...,...,...,...
28307,1,1,1,1,1,1,1,1,1,1
33645,1,1,1,1,1,1,1,1,1,1
34105,1,1,1,1,1,1,1,1,1,1


In [51]:
q_views_explore_raw.sort_values(by=['q_id']).tail(20) # Majority views around mid 20s to 30s

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_score_tier,target,q_title_char_count_bin,q_title_word_count_bin
q_view_count,,,,,,,,,,
42,3877,3877,3877,3877,3877,3877,3877,3877,3877,3877
23,4027,4027,4027,4027,4027,4027,4027,4027,4027,4027
24,4128,4128,4128,4128,4128,4128,4128,4128,4128,4128
41,4131,4131,4131,4131,4131,4131,4131,4131,4131,4131
40,4148,4148,4148,4148,4148,4148,4148,4148,4148,4148
39,4317,4317,4317,4317,4317,4317,4317,4317,4317,4317
38,4521,4521,4521,4521,4521,4521,4521,4521,4521,4521
25,4530,4530,4530,4530,4530,4530,4530,4530,4530,4530
37,4570,4570,4570,4570,4570,4570,4570,4570,4570,4570


In [52]:
# Bin view count for more meaningful analysis
# Establish the bins.
q_view_count_bins = [0,  50, 500, 1000, 5000, 16000]
q_view_count_group_names = ["<50", "50-500", "500-1000", "1000-5000", "5000-16000"]

# Categorize score based on the bins.
posts_questions_df['q_view_count_bin'] = pd.cut(posts_questions_df['q_view_count'], q_view_count_bins, labels=q_view_count_group_names)

In [53]:
# Explore view count
q_views_explore = posts_questions_df.groupby('q_view_count_bin').count()
q_views_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_char_count_bin,q_title_word_count_bin
q_view_count_bin,,,,,,,,,,,
<50,140462,140462,140462,140462,140462,140462,140462,140462,140462,140462,140462
50-500,68310,68310,68310,68310,68310,68310,68310,68308,68310,68310,68310
500-1000,998,998,998,998,998,998,998,998,998,998,998
1000-5000,517,517,517,517,517,517,517,517,517,517,517
5000-16000,66,66,66,66,66,66,66,65,66,66,66


In [54]:
# Readjust bins for more distributed data - add meaningfulness to ML model inputs

# Establish the bins.
q_view_count_bins = [0, 10, 20, 30, 40, 50, 16000]
q_view_count_group_names = ["<10", "10-20", "20-30","30-40", "40-50", "50-16000"]

# Categorize score based on the bins.
posts_questions_df['q_view_count_bin'] = pd.cut(posts_questions_df['q_view_count'], q_view_count_bins, labels=q_view_count_group_names)
# Explore view count
q_views_explore = posts_questions_df.groupby('q_view_count_bin').count()
q_views_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_char_count_bin,q_title_word_count_bin
q_view_count_bin,,,,,,,,,,,
<10,322,322,322,322,322,322,322,322,322,322,322
10-20,14596,14596,14596,14596,14596,14596,14596,14596,14596,14596,14596
20-30,43859,43859,43859,43859,43859,43859,43859,43859,43859,43859,43859
30-40,47165,47165,47165,47165,47165,47165,47165,47165,47165,47165,47165
40-50,34520,34520,34520,34520,34520,34520,34520,34520,34520,34520,34520
50-16000,69891,69891,69891,69891,69891,69891,69891,69888,69891,69891,69891


In [55]:
# Determine word count of body
posts_questions_df['q_body_word_count']= posts_questions_df['q_body'].str.split().str.len() #word count
# Group by body word count
q_body_explore_raw = posts_questions_df.groupby('q_body_word_count').count()
q_body_explore_raw

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin
q_body_word_count,,,,,,,,,,,,
5,3,3,3,3,3,3,3,3,3,3,3,3
6,2,2,2,2,2,2,2,2,2,2,2,2
7,3,3,3,3,3,3,3,3,3,3,3,3
8,6,6,6,6,6,6,6,6,6,6,6,6
9,14,14,14,14,14,14,14,14,14,14,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...
4423,1,1,1,1,1,1,1,1,1,1,1,1
4475,1,1,1,1,1,1,1,1,1,1,1,1
4762,1,1,1,1,1,1,1,1,1,1,1,1


In [56]:
q_body_explore_raw.sort_values(by=['q_id']).tail(20) # Majority views around mid 20s to 30s

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin
q_body_word_count,,,,,,,,,,,,
104,1087,1087,1087,1087,1087,1087,1087,1087,1087,1087,1087,1087
101,1088,1088,1088,1088,1088,1088,1088,1088,1088,1088,1088,1088
79,1093,1093,1093,1093,1093,1093,1093,1093,1093,1093,1093,1093
103,1095,1095,1095,1095,1095,1095,1095,1095,1095,1095,1095,1095
85,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097
86,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097
92,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097,1097
91,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099
95,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1102,1101


In [57]:
# Bin body word count for more meaningful analysis
# Establish the bins.
q_body_len_bins = [0,  100, 500, 10000]
q_body_len_group_names = ["<100", "100-500", "500-1000", "1000-5000", "5000-10000"]

# Categorize body length based on the bins.
posts_questions_df['q_body_len_bin'] = pd.cut(posts_questions_df['q_body_word_count'], q_body_len_bins, labels=q_body_len_group_names)

# Explore body length
q_body_len_explore = posts_questions_df.groupby('q_body_len_bin').count()
q_body_len_explore

ValueError: Bin labels must be one fewer than the number of bin edges

In [58]:
# Readjust bins for more distributed data - add meaningfulness to ML model inputs

# Establish the bins.
q_body_len_bins = [0, 50, 100, 250, 500, 10000]
q_body_len_group_names = ["<50", "50-100", "100-250", "250-500", "500-10000"]

# Categorize body length based on the bins.
posts_questions_df['q_body_len_bin'] = pd.cut(posts_questions_df['q_body_word_count'], q_body_len_bins, labels=q_body_len_group_names)

# Explore body length
q_body_len_explore = posts_questions_df.groupby('q_body_len_bin').count()
q_body_len_explore

,q_id,q_title,q_body,accepted_answer_id,q_creation_date,q_tags,q_view_count,q_score_tier,target,q_title_char_count_bin,q_title_word_count_bin,q_view_count_bin,q_body_word_count
q_body_len_bin,,,,,,,,,,,,,
<50,15444,15444,15444,15444,15444,15444,15444,15443,15444,15444,15444,15443,15444
50-100,51168,51168,51168,51168,51168,51168,51168,51164,51168,51168,51168,51160,51168
100-250,102199,102199,102199,102199,102199,102199,102199,102197,102199,102199,102199,102193,102199
250-500,33277,33277,33277,33277,33277,33277,33277,33276,33277,33277,33277,33277,33277
500-10000,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280,8280


In [59]:
# Create list with all values within q_body
q_body = posts_questions_df['q_body']
q_body

0         <p>sorry in advance for my lack of technical l...
1         <p>Can I get multiple models for a query like ...
2         <p>I'm trying to make a shuffled copy of an ar...
3         <p>I am currently learning python and I was wo...
4         <p>I have install Laravel 8 fresh project. And...
                                ...                        
210363    <p>I'm working on a chat app where users can s...
210364    <p>I'm very new to bash scripting and I'm stuc...
210365    <p>I want to delete all lines that contains an...
210366    <p>I have a time series data of the following ...
210367    <p>i have file in /pdf/<strong>0001_test.pdf</...
Name: q_body, Length: 210368, dtype: object

In [60]:
# Find all unique HTML tags in q_body list
    # Use re library instead of hardcoding things
    # Use functions because it's a repetitive process

# Count number of each unique tag for every row in q_body list
    # For each row, how many <p>, how many <img tags>, etc
    # Store as separate column in q_body
    
# Group by tags


In [61]:
# Create function to count tags
    # Create list from tags column
    # Count number of "|" + 1

In [62]:
# Explore the Votes table
    # Filterby date
    # Filterby Question post
    # Group by
# Expand our query beyond May 2021, go back 12 months

# Create our own tags table and categorize and encode it

In [63]:
# Pull only relevant columns to ML model
# Sample data limited to June 2021, will scale up for next steps
query_job = client.query(
        """
        SELECT
        id, accepted_answer_id, creation_date, tags
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE accepted_answer_id is not null AND creation_date > '2021-05-01'
        ORDER BY accepted_answer_id"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [64]:
cols = ['id', 'accepted_answer_id', 'question_creation_date', 'tags']

In [65]:
post_questions = pd.DataFrame.from_records(results_list, columns=cols)
post_questions

,id,accepted_answer_id,question_creation_date,tags
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3
...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum


In [66]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-01-01'
        ORDER BY id
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [67]:
cols = [field.name for field in results.schema]

In [68]:
post_answers = pd.DataFrame.from_records(results_list, columns=cols)
post_answers

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,65526388,None,<p>Here is one solution. You can first do a gr...,None,None,0,None,2021-01-01 00:00:01.653000+00:00,None,2021-01-01 00:00:01.653000+00:00,NaT,None,NaN,None,4520420,65525964,2,2,None,None
1,65526389,None,<p>Echoing from the individual threads can be ...,None,None,0,None,2021-01-01 00:00:11.187000+00:00,None,2021-01-01 00:00:11.187000+00:00,NaT,None,NaN,None,342497,54217345,2,0,None,None
2,65526391,None,"<p>I have found similar code here (<a href=""ht...",None,None,5,None,2021-01-01 00:00:41.780000+00:00,None,2021-01-01 00:00:41.780000+00:00,NaT,None,NaN,None,13436156,65526354,2,0,None,None
3,65526393,None,"<p>Encountered the same error, just wanna add ...",None,None,0,None,2021-01-01 00:01:25.723000+00:00,None,2021-01-01 00:01:25.723000+00:00,NaT,None,NaN,None,12552972,48184969,2,0,None,None
4,65526394,None,<p>It's difficult to have all three types of v...,None,None,5,None,2021-01-01 00:01:33.950000+00:00,None,2021-01-01 00:01:33.950000+00:00,NaT,None,NaN,None,12602208,65525938,2,0,None,None
5,65526395,None,<p>to be able to change other user passwords y...,None,None,0,None,2021-01-01 00:01:32.977000+00:00,None,2021-01-01 00:01:32.977000+00:00,NaT,None,NaN,None,5562372,65523941,2,0,None,None
6,65526398,None,<blockquote>\n<p>Is it safe (and possible?) to...,None,None,2,None,2021-01-01 00:01:52.123000+00:00,None,2021-01-01 00:09:28.470000+00:00,2021-01-01 00:09:28.470000+00:00,None,14660.0,None,14660,65526234,2,1,None,None
7,65526403,None,<pre><code>def shuffle(deck):\n split_deck ...,None,None,0,None,2021-01-01 00:02:34.570000+00:00,None,2021-01-01 00:02:34.570000+00:00,NaT,None,NaN,None,4718350,65526315,2,1,None,None
8,65526404,None,"<p>FYI, here's a simple app that will do what ...",None,None,1,None,2021-01-01 00:02:40.090000+00:00,None,2021-01-01 00:09:09.673000+00:00,2021-01-01 00:09:09.673000+00:00,None,816620.0,None,816620,65514069,2,0,None,None
9,65526405,None,<p>First you'll want to check if there is no r...,None,None,0,None,2021-01-01 00:02:40.743000+00:00,None,2021-01-01 00:22:23.763000+00:00,2021-01-01 00:22:23.763000+00:00,None,14560865.0,None,14560865,65526331,2,2,None,None


In [69]:
# Pull only relevant columns to ML model
query_job = client.query(
        """
        SELECT
        id, creation_date
        FROM `bigquery-public-data.stackoverflow.posts_answers`
        WHERE creation_date > '2021-05-01'
        ORDER BY id"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]

In [70]:
cols = ['answer_id', 'answer_creation_date']

In [71]:
post_answers = pd.DataFrame.from_records(results_list, columns=cols)
post_answers

,answer_id,answer_creation_date
0,67341731,2021-05-01 00:00:43.057000+00:00
1,67341735,2021-05-01 00:02:14.853000+00:00
2,67341736,2021-05-01 00:02:18.417000+00:00
3,67341740,2021-05-01 00:02:59.713000+00:00
4,67341741,2021-05-01 00:03:03.637000+00:00
...,...,...
698795,69061022,2021-09-05 06:21:38.487000+00:00
698796,69061025,2021-09-05 06:22:13.693000+00:00
698797,69061028,2021-09-05 06:22:52.787000+00:00
698798,69061029,2021-09-05 06:23:42.780000+00:00


In [72]:
post_questions = post_questions.rename(columns={"id":"parent_id"})
post_questions

,parent_id,accepted_answer_id,question_creation_date,tags
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3
...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum


In [73]:
merged_df = pd.merge(post_questions, post_answers, left_on=  ['accepted_answer_id'],
                   right_on= ['answer_id'], 
                   how = 'left')
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_id,answer_creation_date
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,67341801,2021-05-01 00:14:38.340000+00:00
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,67341857,2021-05-01 00:26:39.053000+00:00
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,67341911,2021-05-01 00:36:54.910000+00:00
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,67341961,2021-05-01 00:46:37.893000+00:00
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,67341974,2021-05-01 00:48:52.390000+00:00
...,...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,69060817,2021-09-05 05:39:44.423000+00:00
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,69060824,2021-09-05 05:41:47.300000+00:00
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,69060862,2021-09-05 05:51:12.690000+00:00
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,69060898,2021-09-05 05:58:50.680000+00:00


In [74]:
# Compare accepted_answer_id and answer_id to verify identical
merged_df['accepted_answer_id'].equals(merged_df['answer_id'])

True

In [75]:
# Drop "answer_id" column because redundant
merged_df = merged_df.drop(columns=['answer_id'])
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,2021-05-01 00:14:38.340000+00:00
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,2021-05-01 00:26:39.053000+00:00
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,2021-05-01 00:36:54.910000+00:00
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,2021-05-01 00:46:37.893000+00:00
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,2021-05-01 00:48:52.390000+00:00
...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,2021-09-05 05:39:44.423000+00:00
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,2021-09-05 05:41:47.300000+00:00
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,2021-09-05 05:51:12.690000+00:00
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,2021-09-05 05:58:50.680000+00:00


In [76]:
# Add column for day of question_creation_date [question_day]
merged_df['question_day'] = merged_df['question_creation_date'].dt.day_name()
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,2021-05-01 00:14:38.340000+00:00,Saturday
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,2021-05-01 00:26:39.053000+00:00,Saturday
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,2021-05-01 00:36:54.910000+00:00,Saturday
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,2021-05-01 00:46:37.893000+00:00,Saturday
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,2021-05-01 00:48:52.390000+00:00,Saturday
...,...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,2021-09-05 05:39:44.423000+00:00,Sunday
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,2021-09-05 05:41:47.300000+00:00,Sunday
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,2021-09-05 05:51:12.690000+00:00,Sunday
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,2021-09-05 05:58:50.680000+00:00,Sunday


In [77]:
# Add column for hour value of question_creation_date [question_time]
merged_df['question_hour'] = merged_df['question_creation_date'].dt.hour
merged_df['question_hour_min'] = merged_df['question_creation_date'].dt.strftime('%H:%M')

merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min
0,67341742,67341801,2021-05-01 00:03:03.923000+00:00,javascript|html|node.js|obs,2021-05-01 00:14:38.340000+00:00,Saturday,0,00:03
1,67341817,67341857,2021-05-01 00:17:39.380000+00:00,smt|formal-methods|satisfiability|cvc4,2021-05-01 00:26:39.053000+00:00,Saturday,0,00:17
2,67341895,67341911,2021-05-01 00:32:18.187000+00:00,python|numpy,2021-05-01 00:36:54.910000+00:00,Saturday,0,00:32
3,67341936,67341961,2021-05-01 00:41:17.673000+00:00,python,2021-05-01 00:46:37.893000+00:00,Saturday,0,00:41
4,67341921,67341974,2021-05-01 00:38:52.743000+00:00,laravel-8|vuejs3,2021-05-01 00:48:52.390000+00:00,Saturday,0,00:38
...,...,...,...,...,...,...,...,...
210363,69060773,69060817,2021-09-05 05:31:01.447000+00:00,android-progressbar,2021-09-05 05:39:44.423000+00:00,Sunday,5,05:31
210364,69060793,69060824,2021-09-05 05:34:58.063000+00:00,bash|virtual-machine,2021-09-05 05:41:47.300000+00:00,Sunday,5,05:34
210365,69060850,69060862,2021-09-05 05:45:41.713000+00:00,intellij-idea,2021-09-05 05:51:12.690000+00:00,Sunday,5,05:45
210366,69060801,69060898,2021-09-05 05:36:53.920000+00:00,python|pandas|dataframe|time-series|cumsum,2021-09-05 05:58:50.680000+00:00,Sunday,5,05:36


In [78]:
# Add column for duration till accepted answer (answer_creation_date - question_creation_date)
merged_df['accepted_answer_duration'] = (merged_df['answer_creation_date']-merged_df['question_creation_date'])/np.timedelta64(1,'h')
merged_df = merged_df.sort_values(by='accepted_answer_duration', ascending=False)
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
207957,67420325,69033254,2021-05-06 14:36:35.883000+00:00,windows|code-signing,2021-09-02 15:34:47.857000+00:00,Thursday,14,14:36,2856.969993
202268,67365624,68981014,2021-05-03 08:18:01.683000+00:00,node.js|express|oauth-2.0|discord,2021-08-30 08:27:48.440000+00:00,Monday,8,08:18,2856.162988
210268,67454378,69059067,2021-05-09 04:49:57.043000+00:00,java|spring|spring-boot|validation,2021-09-04 21:28:51.283000+00:00,Sunday,4,04:49,2848.648400
196437,67355370,68931282,2021-05-02 10:37:24.760000+00:00,c++|dll|uwp|dllimport|xbox,2021-08-26 00:54:18.303000+00:00,Sunday,10,10:37,2774.281540
198903,67385222,68952849,2021-05-04 12:57:32.417000+00:00,c++|linux|gcc|cmake|arm,2021-08-27 11:43:25.663000+00:00,Tuesday,12,12:57,2758.764791
...,...,...,...,...,...,...,...,...,...
113654,68262350,68262351,2021-07-05 21:41:04.843000+00:00,javascript|node.js|jestjs|tsc,2021-07-05 21:41:04.843000+00:00,Monday,21,21:41,0.000000
187252,68855357,68855358,2021-08-19 23:10:29.470000+00:00,firebase|google-cloud-functions,2021-08-19 23:10:29.470000+00:00,Thursday,23,23:10,0.000000
195492,68923827,68923828,2021-08-25 13:27:06.257000+00:00,questdb,2021-08-25 13:27:06.257000+00:00,Wednesday,13,13:27,0.000000
44851,67708855,67708856,2021-05-26 16:14:40.067000+00:00,opengl|3d|phong,2021-05-26 16:14:40.067000+00:00,Wednesday,16,16:14,0.000000


In [79]:
# Additional method to verify number of rows with zero response time (what's the possible explanation?)
zero_time = merged_df.groupby('accepted_answer_duration').count()
zero_time 

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min
accepted_answer_duration,,,,,,,,
0.000000,930,930,930,930,930,930,930,930
0.000166,1,1,1,1,1,1,1,1
0.000191,1,1,1,1,1,1,1,1
0.000194,1,1,1,1,1,1,1,1
0.003704,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
2758.764791,1,1,1,1,1,1,1,1
2774.281540,1,1,1,1,1,1,1,1
2848.648400,1,1,1,1,1,1,1,1


In [80]:
# drop data rows where duration is 0
indexNames = merged_df[ merged_df['accepted_answer_duration'] == 0.000000 ].index
merged_df.drop(indexNames, inplace=True)
merged_df

,parent_id,accepted_answer_id,question_creation_date,tags,answer_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
207957,67420325,69033254,2021-05-06 14:36:35.883000+00:00,windows|code-signing,2021-09-02 15:34:47.857000+00:00,Thursday,14,14:36,2856.969993
202268,67365624,68981014,2021-05-03 08:18:01.683000+00:00,node.js|express|oauth-2.0|discord,2021-08-30 08:27:48.440000+00:00,Monday,8,08:18,2856.162988
210268,67454378,69059067,2021-05-09 04:49:57.043000+00:00,java|spring|spring-boot|validation,2021-09-04 21:28:51.283000+00:00,Sunday,4,04:49,2848.648400
196437,67355370,68931282,2021-05-02 10:37:24.760000+00:00,c++|dll|uwp|dllimport|xbox,2021-08-26 00:54:18.303000+00:00,Sunday,10,10:37,2774.281540
198903,67385222,68952849,2021-05-04 12:57:32.417000+00:00,c++|linux|gcc|cmake|arm,2021-08-27 11:43:25.663000+00:00,Tuesday,12,12:57,2758.764791
...,...,...,...,...,...,...,...,...,...
119495,68309670,68309676,2021-07-08 23:38:35.807000+00:00,java|totp|apache-commons-codec,2021-07-08 23:38:52.473000+00:00,Thursday,23,23:38,0.004629
205921,69012861,69012865,2021-09-01 11:27:42.277000+00:00,php|laravel|laravel-6,2021-09-01 11:27:55.613000+00:00,Wednesday,11,11:27,0.003704
171983,68730474,68730475,2021-08-10 16:30:19.920000+00:00,palantir-foundry|foundry-code-workbooks,2021-08-10 16:30:20.620000+00:00,Tuesday,16,16:30,0.000194
166581,68686714,68686715,2021-08-06 19:34:02.003000+00:00,palantir-foundry|foundry-slate,2021-08-06 19:34:02.690000+00:00,Friday,19,19:34,0.000191


In [81]:
# create a subset of merged_df for ML model
#response_duration_ML_df = merged_df['accepted_answer_id', 'question_day', 'question_hour', 'question_hour_min', 'accepted_answer_duration']
#response_duration_ML_df

practice_ML_df = merged_df[['accepted_answer_id', 'question_creation_date', 'question_day', 'question_hour', 'question_hour_min', 'accepted_answer_duration']]
practice_ML_df

,accepted_answer_id,question_creation_date,question_day,question_hour,question_hour_min,accepted_answer_duration
207957,69033254,2021-05-06 14:36:35.883000+00:00,Thursday,14,14:36,2856.969993
202268,68981014,2021-05-03 08:18:01.683000+00:00,Monday,8,08:18,2856.162988
210268,69059067,2021-05-09 04:49:57.043000+00:00,Sunday,4,04:49,2848.648400
196437,68931282,2021-05-02 10:37:24.760000+00:00,Sunday,10,10:37,2774.281540
198903,68952849,2021-05-04 12:57:32.417000+00:00,Tuesday,12,12:57,2758.764791
...,...,...,...,...,...,...
119495,68309676,2021-07-08 23:38:35.807000+00:00,Thursday,23,23:38,0.004629
205921,69012865,2021-09-01 11:27:42.277000+00:00,Wednesday,11,11:27,0.003704
171983,68730475,2021-08-10 16:30:19.920000+00:00,Tuesday,16,16:30,0.000194
166581,68686715,2021-08-06 19:34:02.003000+00:00,Friday,19,19:34,0.000191


In [ ]:
# Reorder columns

In [82]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.votes`
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]
cols = [field.name for field in results.schema]
votes = pd.DataFrame.from_records(results_list, columns=cols)
votes

,id,creation_date,post_id,vote_type_id
0,90049416,2015-05-27 00:00:00+00:00,26697261,10
1,90051492,2015-05-27 00:00:00+00:00,3088012,2
2,90051531,2015-05-27 00:00:00+00:00,5783020,2
3,90051719,2015-05-27 00:00:00+00:00,28254932,2
4,90051947,2015-05-27 00:00:00+00:00,30490441,2
5,90054381,2015-05-27 00:00:00+00:00,30471183,1
6,90054453,2015-05-27 00:00:00+00:00,30491199,2
7,90054785,2015-05-27 00:00:00+00:00,14397716,2
8,90055207,2015-05-27 00:00:00+00:00,21331988,2
9,90058648,2015-05-27 00:00:00+00:00,19139095,2


In [83]:
query_job = client.query(
        """
        SELECT
        *
        FROM `bigquery-public-data.stackoverflow.votes`
        LIMIT 10"""
    )

results = query_job.result()  # Waits for job to complete.
    # [END bigquery_simple_app_query]

    # [START bigquery_simple_app_print]
# for row in results:
#     print(row)
results_list = [row for row in results]
cols = [field.name for field in results.schema]
votes = pd.DataFrame.from_records(results_list, columns=cols)
votes

,id,creation_date,post_id,vote_type_id
0,90049416,2015-05-27 00:00:00+00:00,26697261,10
1,90051492,2015-05-27 00:00:00+00:00,3088012,2
2,90051531,2015-05-27 00:00:00+00:00,5783020,2
3,90051719,2015-05-27 00:00:00+00:00,28254932,2
4,90051947,2015-05-27 00:00:00+00:00,30490441,2
5,90054381,2015-05-27 00:00:00+00:00,30471183,1
6,90054453,2015-05-27 00:00:00+00:00,30491199,2
7,90054785,2015-05-27 00:00:00+00:00,14397716,2
8,90055207,2015-05-27 00:00:00+00:00,21331988,2
9,90058648,2015-05-27 00:00:00+00:00,19139095,2
